# 1 - BiLSTM for PoS Tagging

## Introduction

In this series we'll be building a machine learning model that produces an output for every element in an input sequence, using PyTorch and TorchText. Specifically, we will be inputting a sequence of text and the model will output a part-of-speech (PoS) tag for each token in the input text. This can also be used for named entity recognition (NER), where the output for each token will be what type of entity, if any, the token is.

In this notebook, we'll be implementing a multi-layer bi-directional LSTM (BiLSTM) to predict PoS tags using the Universal Dependencies English Web Treebank (UDPOS) dataset.

## Preparing Data

First, let's import the necessary Python modules.

In [19]:
!pip install transformers



In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy import data
from torchtext.legacy import datasets

import spacy
import numpy as np
import pandas as pd

import time
import random

In [23]:
from google.colab import drive
drive.mount('/content/gdrive/')


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


Next, we'll set the random seeds for reproducability.

In [24]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

One of the key parts of TorchText is the `Field`. The `Field` handles how your dataset is processed.

Our `TEXT` field handles how the text that we need to tag is dealt with. All we do here is set `lower = True` which lowercases all of the text.

Next we'll define the `Fields` for the tags. This dataset actually has two different sets of tags, [universal dependency (UD) tags](https://universaldependencies.org/u/pos/) and [Penn Treebank (PTB) tags](https://www.sketchengine.eu/penn-treebank-tagset/). We'll only train our model on the UD tags, but will load the PTB tags to show how they could be used instead.

`UD_TAGS` handles how the UD tags should be handled. Our `TEXT` vocabulary - which we'll build later - will have *unknown* tokens in it, i.e. tokens that are not within our vocabulary. However, we won't have unknown tags as we are dealing with a finite set of possible tags. TorchText `Fields` initialize a default unknown token, `<unk>`, which we remove by setting `unk_token = None`.

`PTB_TAGS` does the same as `UD_TAGS`, but handles the PTB tags instead.

In [25]:
TEXT = data.Field(lower = True)
PTB_TAGS = data.Field(unk_token = None)


We then define `fields`, which handles passing our fields to the dataset.

Note that order matters, if you only wanted to load the PTB tags your field would be:

```
fields = (("text", TEXT), (None, None), ("ptbtags", PTB_TAGS))
```

Where `None` tells TorchText to not load those tags.

In [26]:
#fields = (("text", TEXT), ("udtags", UD_TAGS))
fields = (("text", TEXT), ("ptbtags", PTB_TAGS))

In [27]:
data_list = []
with open('/content/gdrive/My Drive/Colab Notebooks/ontonotes.tsv', 'r') as f:
  data_f = f.readlines()
  single_sentence = ''
  label = ''
  for i in data_f:
    sentence = i.strip().split('\t')
    #print(sentence[0])
    if sentence[0] == '*':
      continue
    if sentence[1] != '.':
      single_sentence += sentence[1] + ' '
      #print(single_sentence)
      label += sentence[2] + ' '
      #print(label)
    if sentence[1] == '.':
      data_list.append((single_sentence.strip(), label.strip()))
      single_sentence = ''
      label = ''
print(data_list[:3])

[('A Saudi woman issues a voice recording in which she mourns her brother and demands the punishment of his killers', 'DT JJ NN VBZ DT NN NN IN WDT PRP VBZ PRP$ NN CC VBZ DT NN IN PRP$ NNS'), ('HAD1999 In a unique way , a young Saudi woman has resorted to issuing a sound tape in which she mourns her brother murdered in Hafr Al Batin , in the hope of rekindling interest in his case , in which 3 young men were accused of killing him and which has not so far been resolved', 'NNP IN DT JJ NN , DT JJ JJ NN VBZ VBN IN VBG DT NN NN IN WDT PRP VBZ PRP$ NN VBN IN NNP NNP NNP , IN DT NN IN VBG NN IN PRP$ NN , IN WDT CD JJ NNS VBD VBN IN VBG PRP CC WDT VBZ RB RB RB VBN VBN'), ("The tape contains excerpts of poetry , dramatic voice recitals , detailed accounts of the young man 's murder circumstances of the background to the investigation of the case for which the young woman is demanding expedited proceedings and punishment of those involved", 'DT NN VBZ NNS IN NN , JJ NN NNS , JJ NNS IN DT JJ NN

In [28]:
examples = list(map(lambda x: data.example.Example.fromlist(list(x), fields=fields), data_list))

train_data = data.Dataset(examples[:45000], fields=fields)
valid_data = data.Dataset(examples[45000:50000], fields=fields)
test_data = data.Dataset(examples[50000:], fields=fields)

Next, we load the UDPOS dataset using our defined fields.

We can check how many examples are in each section of the dataset by checking their length.

In [29]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 45000
Number of validation examples: 5000
Number of testing examples: 6519


Let's print out an example:

In [30]:
print(vars(train_data.examples[0]))

{'text': ['a', 'saudi', 'woman', 'issues', 'a', 'voice', 'recording', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'and', 'demands', 'the', 'punishment', 'of', 'his', 'killers'], 'ptbtags': ['DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'WDT', 'PRP', 'VBZ', 'PRP$', 'NN', 'CC', 'VBZ', 'DT', 'NN', 'IN', 'PRP$', 'NNS']}


We can also view the text and tags separately:

In [31]:
print(vars(train_data.examples[0])['text'])

['a', 'saudi', 'woman', 'issues', 'a', 'voice', 'recording', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'and', 'demands', 'the', 'punishment', 'of', 'his', 'killers']


In [32]:
print(vars(train_data.examples[0])['ptbtags'])

['DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'WDT', 'PRP', 'VBZ', 'PRP$', 'NN', 'CC', 'VBZ', 'DT', 'NN', 'IN', 'PRP$', 'NNS']


Next, we'll build the vocabulary - a mapping of tokens to integers. 

We want some unknown tokens within our dataset in order to replicate how this model would be used in real life, so we set the `min_freq` to 2 which means only tokens that appear twice in the training set will be added to the vocabulary and the rest will be replaced by `<unk>` tokens.

We also load the [GloVe](https://nlp.stanford.edu/projects/glove/) pre-trained token embeddings. Specifically, the 100-dimensional embeddings that have been trained on 6 billion tokens. Using pre-trained embeddings usually leads to improved performance - although admittedly the dataset used in this tutorial is too small to take advantage of the pre-trained embeddings. 

`unk_init` is used to initialize the token embeddings which are not in the pre-trained embedding vocabulary. By default this sets those embeddings to zeros, however it is better to not have them all initialized to the same value, so we initialize them from a Normal/Gaussian distribution.

These pre-trained vectors are now loaded into our vocabulary and we will initialize our model with these values later.

In [33]:
MIN_FREQ = 2

TEXT.build_vocab(train_data, 
                 min_freq = MIN_FREQ)


PTB_TAGS.build_vocab(train_data)


In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModel.from_pretrained("bert-base-cased")
model.eval()

def get_bert_emb(word):
  encoded_input = tokenizer(word, padding=True, truncation=True, max_length=128, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)

      return model_output[0][0][1].numpy()

vocab_df = pd.DataFrame({'vocab':list(TEXT.vocab.stoi.keys())})
vocab_df['emb'] = vocab_df['vocab'].apply(get_bert_emb)
vocab_df.to_pickle("vocab_emb")

In [35]:
vocab_df = pd.read_pickle("vocab_emb")
our_vectors = vocab_df['emb'].tolist()
our_vectors = torch.Tensor(our_vectors)

TEXT.vocab.vectors = our_vectors

our_vectors.shape
vocab_df.head()

,vocab,emb
0,<unk>,"[0.10041629, 0.21012346, 0.16489458, -0.108804..."
1,<pad>,"[-0.020771498, 0.32556504, 0.059507508, -0.014..."
2,the,"[0.5092548, -0.4575306, 0.32636842, -0.4087785..."
3,",","[0.487278, -0.37566534, 0.41073442, -0.3688913..."
4,of,"[0.64068377, -0.28513005, 0.12464522, -0.05887..."


We can check how many tokens and tags are in our vocabulary by getting their length:

In [36]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in PTB_TAGS vocabulary: {len(PTB_TAGS.vocab)}")

Unique tokens in TEXT vocabulary: 23221
Unique tokens in PTB_TAGS vocabulary: 50


Exploring the vocabulary, we can check the most common tokens within our texts:

In [37]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 65953), (',', 56119), ('of', 31192), ('to', 28283), ('and', 26983), ('a', 22699), ('in', 21868), ('that', 13682), ('is', 12048), ("'s", 11001), ('-', 10852), ('for', 9815), ('it', 8879), ('on', 7303), ('/.', 7165), ('with', 6603), ('as', 6069), ('this', 5879), ('was', 5859), ('are', 5784)]


We can see the vocabularies for both of our tags:

In [38]:
print(PTB_TAGS.vocab.itos)

['<pad>', 'NN', 'IN', 'DT', 'NNP', 'JJ', 'NNS', ',', 'RB', 'PRP', 'VB', 'CC', 'VBD', 'VBZ', 'CD', 'VBN', 'VBP', 'VBG', 'TO', 'MD', 'PRP$', '.', 'HYPH', 'POS', "''", '``', 'WDT', 'UH', 'WP', ':', 'RP', 'WRB', 'NNPS', 'JJR', '$', 'EX', 'JJS', '-RRB-', '-LRB-', 'RBR', 'XX', 'PDT', 'RBS', 'FW', 'NFP', 'SYM', 'LS', 'WP$', 'ADD', 'AFX']


We can also see how many of each tag are in our vocabulary:

In [39]:
print(PTB_TAGS.vocab.freqs.most_common())

[('NN', 166558), ('IN', 137756), ('DT', 110923), ('NNP', 103729), ('JJ', 76184), ('NNS', 67555), (',', 56740), ('RB', 50828), ('PRP', 39402), ('VB', 38789), ('CC', 35281), ('VBD', 32519), ('VBZ', 31577), ('CD', 26735), ('VBN', 26419), ('VBP', 23064), ('VBG', 22114), ('TO', 18006), ('MD', 13413), ('PRP$', 12240), ('.', 10836), ('HYPH', 10164), ('POS', 8493), ("''", 6438), ('``', 6405), ('WDT', 5842), ('UH', 5158), ('WP', 4848), (':', 4684), ('RP', 4573), ('WRB', 4285), ('NNPS', 3744), ('JJR', 3591), ('$', 2310), ('EX', 2174), ('JJS', 2071), ('-RRB-', 2025), ('-LRB-', 1999), ('RBR', 1804), ('XX', 1189), ('PDT', 859), ('RBS', 769), ('FW', 626), ('NFP', 437), ('SYM', 319), ('LS', 230), ('WP$', 188), ('ADD', 181), ('AFX', 11)]


We can also view how common each of the tags are within the training set:

In [40]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
        
    return tag_counts_percentages

In [41]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(PTB_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

NN		166558		14.0%
IN		137756		11.6%
DT		110923		 9.4%
NNP		103729		 8.7%
JJ		76184		 6.4%
NNS		67555		 5.7%
,		56740		 4.8%
RB		50828		 4.3%
PRP		39402		 3.3%
VB		38789		 3.3%
CC		35281		 3.0%
VBD		32519		 2.7%
VBZ		31577		 2.7%
CD		26735		 2.3%
VBN		26419		 2.2%
VBP		23064		 1.9%
VBG		22114		 1.9%
TO		18006		 1.5%
MD		13413		 1.1%
PRP$		12240		 1.0%
.		10836		 0.9%
HYPH		10164		 0.9%
POS		8493		 0.7%
''		6438		 0.5%
``		6405		 0.5%
WDT		5842		 0.5%
UH		5158		 0.4%
WP		4848		 0.4%
:		4684		 0.4%
RP		4573		 0.4%
WRB		4285		 0.4%
NNPS		3744		 0.3%
JJR		3591		 0.3%
$		2310		 0.2%
EX		2174		 0.2%
JJS		2071		 0.2%
-RRB-		2025		 0.2%
-LRB-		1999		 0.2%
RBR		1804		 0.2%
XX		1189		 0.1%
PDT		859		 0.1%
RBS		769		 0.1%
FW		626		 0.1%
NFP		437		 0.0%
SYM		319		 0.0%
LS		230		 0.0%
WP$		188		 0.0%
ADD		181		 0.0%
AFX		11		 0.0%


The final part of data preparation is handling the iterator. 

This will be iterated over to return batches of data to process. Here, we set the batch size and the `device` - which is used to place the batches of tensors on our GPU, if we have one. 

In [42]:
BATCH_SIZE = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,
    sort=False)

cuda


## Building the Model

Next up, we define our model - a multi-layer bi-directional LSTM. The image below shows a simplified version of the model with only one LSTM layer and omitting the LSTM's cell state for clarity.

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bidirectional-lstm.png?raw=1)

The model takes in a sequence of tokens, $X = \{x_1, x_2,...,x_T\}$, passes them through an embedding layer, $e$, to get the token embeddings, $e(X) = \{e(x_1), e(x_2), ..., e(x_T)\}$.

These embeddings are processed - one per time-step - by the forward and backward LSTMs. The forward LSTM processes the sequence from left-to-right, whilst the backward LSTM processes the sequence right-to-left, i.e. the first input to the forward LSTM is $x_1$ and the first input to the backward LSTM is $x_T$. 

The LSTMs also take in the the hidden, $h$, and cell, $c$, states from the previous time-step

$$h^{\rightarrow}_t = \text{LSTM}^{\rightarrow}(e(x^{\rightarrow}_t), h^{\rightarrow}_{t-1}, c^{\rightarrow}_{t-1})$$
$$h^{\leftarrow}_t=\text{LSTM}^{\leftarrow}(e(x^{\leftarrow}_t), h^{\leftarrow}_{t-1}, c^{\leftarrow}_{t-1})$$

After the whole sequence has been processed, the hidden and cell states are then passed to the next layer of the LSTM.

The initial hidden and cell states, $h_0$ and $c_0$, for each direction and layer are initialized to a tensor full of zeros.

We then concatenate both the forward and backward hidden states from the final layer of the LSTM, $H = \{h_1, h_2, ... h_T\}$, where $h_1 = [h^{\rightarrow}_1;h^{\leftarrow}_T]$, $h_2 = [h^{\rightarrow}_2;h^{\leftarrow}_{T-1}]$, etc. and pass them through a linear layer, $f$, which is used to make the prediction of which tag applies to this token, $\hat{y}_t = f(h_t)$.

When training the model, we will compare our predicted tags, $\hat{Y}$ against the actual tags, $Y$, to calculate a loss, the gradients w.r.t. that loss, and then update our parameters.

We implement the model detailed above in the `BiLSTMPOSTagger` class.

`nn.Embedding` is an embedding layer and the input dimension should be the size of the input (text) vocabulary. We tell it what the index of the padding token is so it does not update the padding token's embedding entry.

`nn.LSTM` is the LSTM. We apply dropout as regularization between the layers, if we are using more than one.

`nn.Linear` defines the linear layer to make predictions using the LSTM outputs. We double the size of the input if we are using a bi-directional LSTM. The output dimensions should be the size of the tag vocabulary.

We also define a dropout layer with `nn.Dropout`, which we use in the `forward` method to apply dropout to the embeddings and the outputs of the final layer of the LSTM.

In [43]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim, 
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_dim, 
                            num_layers = n_layers, 
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)
        
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        #pass text through embedding layer
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pass embeddings into LSTM
        # outputs, (hidden, cell) = self.lstm(embedded)
        outputs, _ = self.lstm(embedded)
        
        #outputs holds the backward and forward hidden states in the final layer
        #hidden and cell are the backward and forward hidden and cell states at the final time-step
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]
        
        #we use our outputs to make a prediction of what the tag should be
        predictions = self.fc(self.dropout(outputs))
        
        #predictions = [sent len, batch size, output dim]
        
        return predictions

## Training the Model

Next, we instantiate the model. We need to ensure the embedding dimensions matches that of the GloVe embeddings we loaded earlier.

The rest of the hyperparmeters have been chosen as sensible defaults, though there may be a combination that performs better on this model and dataset.

The input and output dimensions are taken directly from the lengths of the respective vocabularies. The padding index is obtained using the vocabulary and the `Field` of the text.

In [44]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 768
HIDDEN_DIM = 128
OUTPUT_DIM = len(PTB_TAGS.vocab)
N_LAYERS = 1
BIDIRECTIONAL = False
DROPOUT = 0
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT, 
                        PAD_IDX)

We initialize the weights from a simple Normal distribution. Again, there may be a better initialization scheme for this model and dataset.

In [45]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)
        
model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(23221, 768, padding_idx=1)
  (lstm): LSTM(768, 128)
  (fc): Linear(in_features=128, out_features=50, bias=True)
  (dropout): Dropout(p=0, inplace=False)
)

Next, a small function to tell us how many parameters are in our model. Useful for comparing different models.

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 18,299,954 trainable parameters


We'll now initialize our model's embedding layer with the pre-trained embedding values we loaded earlier.

This is done by getting them from the vocab's `.vectors` attribute and then performing a `.copy` to overwrite the embedding layer's current weights.

In [47]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([23221, 768])


In [48]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.1004,  0.2101,  0.1649,  ...,  0.3756, -0.3962,  0.6146],
        [-0.0208,  0.3256,  0.0595,  ...,  0.0843, -0.2711,  0.7543],
        [ 0.5093, -0.4575,  0.3264,  ..., -0.1868,  0.0561,  0.2649],
        ...,
        [ 0.1004, -0.1152, -0.2222,  ..., -0.2833,  0.4196,  1.0131],
        [-0.1719, -0.6915,  0.8222,  ...,  0.0559,  0.4705,  0.4590],
        [-0.1138, -0.4025,  0.2471,  ...,  0.1777,  0.1432,  0.5662]])

It's common to initialize the embedding of the pad token to all zeros. This, along with setting the `padding_idx` in the model's embedding layer, means that the embedding should always output a tensor full of zeros when a pad token is input.

In [49]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.1004,  0.2101,  0.1649,  ...,  0.3756, -0.3962,  0.6146],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5093, -0.4575,  0.3264,  ..., -0.1868,  0.0561,  0.2649],
        ...,
        [ 0.1004, -0.1152, -0.2222,  ..., -0.2833,  0.4196,  1.0131],
        [-0.1719, -0.6915,  0.8222,  ...,  0.0559,  0.4705,  0.4590],
        [-0.1138, -0.4025,  0.2471,  ...,  0.1777,  0.1432,  0.5662]])


We then define our optimizer, used to update our parameters w.r.t. their gradients. We use Adam with the default learning rate.

In [50]:
optimizer = optim.Adam(model.parameters())

Next, we define our loss function, cross-entropy loss.

Even though we have no `<unk>` tokens within our tag vocab, we still have `<pad>` tokens. This is because all sentences within a batch need to be the same size. However, we don't want to calculate the loss when the target is a `<pad>` token as we aren't training our model to recognize padding tokens.

We handle this by setting the `ignore_index` in our loss function to the index of the padding token in our tag vocabulary.

In [51]:
TAG_PAD_IDX = PTB_TAGS.vocab.stoi[PTB_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

We then place our model and loss function on our GPU, if we have one.

In [52]:
model = model.to(device)
criterion = criterion.to(device)

We will be using the loss value between our predicted and actual tags to train the network, but ideally we'd like a more interpretable way to see how well our model is doing - accuracy.

The issue is that we don't want to calculate accuracy over the `<pad>` tokens as we aren't interested in predicting them.

The function below only calculates accuracy over non-padded tokens. `non_pad_elements` is a tensor containing the indices of the non-pad tokens within an input batch. We then compare the predictions of those elements with the labels to get a count of how many predictions were correct. We then divide this by the number of non-pad elements to get our accuracy value over the batch.

In [53]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).cuda()

Next is the function that handles training our model.

We first set the model to `train` mode to turn on dropout/batch-norm/etc. (if used). Then we iterate over our iterator, which returns a batch of examples. 

For each batch: 
- we zero the gradients over the parameters from the last gradient calculation
- insert the batch of text into the model to get predictions
- as PyTorch loss functions cannot handle 3-dimensional predictions we reshape our predictions
- calculate the loss and accuracy between the predicted tags and actual tags
- call `backward` to calculate the gradients of the parameters w.r.t. the loss
- take an optimizer `step` to update the parameters
- add to the running total of loss and accuracy

In [54]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        text = batch.text
        tags = batch.ptbtags

        
        optimizer.zero_grad()
        
        #text = [sent len, batch size]
        
        predictions = model(text)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        
        loss = criterion(predictions, tags)
                
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

The `evaluate` function is similar to the `train` function, except with changes made so we don't update the model's parameters.

`model.eval()` is used to put the model in evaluation mode, so dropout/batch-norm/etc. are turned off. 

The iteration loop is also wrapped in `torch.no_grad` to ensure we don't calculate any gradients. We also don't need to call `optimizer.zero_grad()` and `optimizer.step()`.

In [55]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text = batch.text
            tags = batch.ptbtags
            
            predictions = model(text)
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Next, we have a small function that tells us how long an epoch takes.

In [56]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model!

After each epoch we check if our model has achieved the best validation loss so far. If it has then we save the parameters of this model and we will use these "best" parameters to calculate performance over our test set.

In [57]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)
print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.303 | Train Acc: 90.98%
	 Val. Loss: 0.229 |  Val. Acc: 92.31%
Epoch: 02 | Epoch Time: 0m 25s
	Train Loss: 0.162 | Train Acc: 94.44%
	 Val. Loss: 0.222 |  Val. Acc: 92.62%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.139 | Train Acc: 95.20%
	 Val. Loss: 0.218 |  Val. Acc: 92.94%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.122 | Train Acc: 95.76%
	 Val. Loss: 0.224 |  Val. Acc: 92.82%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss: 0.107 | Train Acc: 96.28%
	 Val. Loss: 0.236 |  Val. Acc: 92.78%
Epoch: 06 | Epoch Time: 0m 25s
	Train Loss: 0.095 | Train Acc: 96.70%
	 Val. Loss: 0.242 |  Val. Acc: 92.64%
Epoch: 07 | Epoch Time: 0m 25s
	Train Loss: 0.085 | Train Acc: 97.07%
	 Val. Loss: 0.253 |  Val. Acc: 92.73%
Epoch: 08 | Epoch Time: 0m 25s
	Train Loss: 0.077 | Train Acc: 97.36%
	 Val. Loss: 0.265 |  Val. Acc: 92.63%
Epoch: 09 | Epoch Time: 0m 25s
	Train Loss: 0.069 | Train Acc: 97.64%
	 Val. Loss: 0.273 |  Val. Acc: 92.68%
Epoch: 10 | Epoch T

We then load our "best" parameters and evaluate 
performance on the test set.

In [58]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.207 |  Test Acc: 93.12%


## Inference

88% accuracy looks pretty good, but let's see our model tag some actual sentences.

We define a `tag_sentence` function that will:
- put the model into evaluation mode
- tokenize the sentence with spaCy if it is not a list
- lowercase the tokens if the `Field` did
- numericalize the tokens using the vocabulary
- find out which tokens are not in the vocabulary, i.e. are `<unk>` tokens
- convert the numericalized tokens into a tensor and add a batch dimension
- feed the tensor into the model
- get the predictions over the sentence
- convert the predictions into readable tags

As well as returning the tokens and tags, it also returns which tokens were `<unk>` tokens.

In [59]:
def tag_sentence(model, device, sentence, text_field, tag_field):
    
    model.eval()
    
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]
        
    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]

    unk_idx = text_field.vocab.stoi[text_field.unk_token]
    
    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]
    
    token_tensor = torch.LongTensor(numericalized_tokens)
    
    token_tensor = token_tensor.unsqueeze(-1).to(device)
         
    predictions = model(token_tensor)
    
    top_predictions = predictions.argmax(-1)
    
    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]
    
    return tokens, predicted_tags, unks

We'll get an already tokenized example from the training set and test our model's performance.

In [60]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['ptbtags']

print(sentence)

['had1999', 'in', 'a', 'unique', 'way', ',', 'a', 'young', 'saudi', 'woman', 'has', 'resorted', 'to', 'issuing', 'a', 'sound', 'tape', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'murdered', 'in', 'hafr', 'al', 'batin', ',', 'in', 'the', 'hope', 'of', 'rekindling', 'interest', 'in', 'his', 'case', ',', 'in', 'which', '3', 'young', 'men', 'were', 'accused', 'of', 'killing', 'him', 'and', 'which', 'has', 'not', 'so', 'far', 'been', 'resolved']


We can then use our `tag_sentence` function to get the tags. Notice how the tokens referring to subject of the sentence, the "respected cleric", are both `<unk>` tokens!

In [61]:
tokens, pred_tags, unks = tag_sentence(model, 
                                       device, 
                                       sentence, 
                                       TEXT, 
                                       PTB_TAGS)

print(unks)

['had1999', 'rekindling']


We can then check how well it did. Surprisingly, it got every token correct, including the two that were unknown tokens!

In [62]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

NNP		NNP		✔		had1999
IN		IN		✔		in
DT		DT		✔		a
JJ		JJ		✔		unique
NN		NN		✔		way
,		,		✔		,
DT		DT		✔		a
JJ		JJ		✔		young
JJ		JJ		✔		saudi
NN		NN		✔		woman
VBZ		VBZ		✔		has
VBN		VBN		✔		resorted
IN		IN		✔		to
VBG		VBG		✔		issuing
DT		DT		✔		a
NN		NN		✔		sound
NN		NN		✔		tape
IN		IN		✔		in
WDT		WDT		✔		which
PRP		PRP		✔		she
VBZ		VBZ		✔		mourns
PRP$		PRP$		✔		her
NN		NN		✔		brother
VBN		VBN		✔		murdered
IN		IN		✔		in
NNP		NNP		✔		hafr
NNP		NNP		✔		al
NNP		NNP		✔		batin
,		,		✔		,
IN		IN		✔		in
DT		DT		✔		the
NN		NN		✔		hope
IN		IN		✔		of
VBG		VBG		✔		rekindling
NN		NN		✔		interest
IN		IN		✔		in
PRP$		PRP$		✔		his
NN		NN		✔		case
,		,		✔		,
IN		IN		✔		in
WDT		WDT		✔		which
CD		CD		✔		3
JJ		JJ		✔		young
NNS		NNS		✔		men
VBD		VBD		✔		were
VBN		VBN		✔		accused
IN		IN		✔		of
VBG		VBG		✔		killing
PRP		PRP		✔		him
CC		CC		✔		and
WDT		WDT		✔		which
VBZ		VBZ		✔		has
RB		RB		✔		not
RB		RB		✔		so
RB		RB		✔		far
VBN		VBN		✔		been
VBN		VBN		✔		resolved


Let's now make up our own sentence and see how well the model does.

Our example sentence below has every token within the model's vocabulary.

In [63]:
sentence = 'Last semester I participated in "Neural Netwoks implementation and application" course, it was very interesting.'

tokens, tags, unks = tag_sentence(model, 
                                  device, 
                                  sentence, 
                                  TEXT, 
                                 PTB_TAGS)

print(unks)

['neural', 'netwoks', '.']


Looking at the sentence it seems like it gave sensible tags to every token!

In [64]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

JJ		last
NN		semester
PRP		i
VBD		participated
IN		in
``		"
NNP		neural
NNP		netwoks
NN		implementation
CC		and
NN		application
``		"
NN		course
,		,
PRP		it
VBD		was
RB		very
JJ		interesting
NN		.


We've now seen how to implement PoS tagging with PyTorch and TorchText! 

The BiLSTM isn't a state-of-the-art model, in terms of performance, but is a strong baseline for PoS tasks and is a good tool to have in your arsenal.